# So far, the code below which comes from V1 notebook, logs data from FED3 via RbPi board, and sends TTL to the board (to TDT in theory) , in the next step, 

### First I want to make RbPi automatically get data from FEDs as soon as it is switched on (instead of opening the terminal and jupyter lab and running the code)
### Second, I will configure the rbpi pins and the python code to handle 4 FEDs at once

In [ ]:
import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for the TTL output (same as your original setup)
gpio_pins = {
    "LeftPoke": 17,          # Pin for Left poke event (GPIO 17, Physical Pin 11)
    "RightPoke": 27,         # Pin for Right poke event (GPIO 27, Physical Pin 13)
    "LeftWithPellet": 22,    # Pin for Left poke with pellet event (GPIO 22, Physical Pin 15)
    "RightWithPellet": 23,   # Pin for Right poke with pellet event (GPIO 23, Physical Pin 16)
    "Pellet": 24,            # Pin for PelletInWell and PelletTaken (GPIO 24, Physical Pin 18)
}

# Set all pins as output and initially set them to LOW
for pin in gpio_pins.values():
    GPIO.setup(pin, GPIO.OUT)
    GPIO.output(pin, GPIO.LOW)

# Track the state of Pellet in Well
pellet_in_well = False  # Keeps track of whether a pellet is in the well

# Define the column headers based on your desired CSV structure
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Function to send TTL pulse for regular poke events
def send_ttl_signal(pin):
    print(f"Sending TTL signal to pin {pin}")
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.01)  # Send a 10 ms pulse
    GPIO.output(pin, GPIO.LOW)

# Function to handle the PelletInWell/PelletTaken logic
def handle_pellet_event(event_type):
    global pellet_in_well
    if event_type == "Pellet":
        if pellet_in_well:
            GPIO.output(gpio_pins["Pellet"], GPIO.LOW)  # Pellet taken, turn the signal off
            print("Pellet taken, signal turned OFF.")
            pellet_in_well = False  # Update state
            send_ttl_signal(gpio_pins["Pellet"])  # Send a short TTL pulse for PelletTaken
        else:
            print("No pellet was in the well, no signal for pellet taken.")
    elif event_type == "PelletInWell":
        GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)  # Pellet in well, keep signal on
        pellet_in_well = True
        print("Pellet dispensed in well, signal ON.")

# Function to process each event and send TTLs accordingly
def process_event(event_type, timestamp):
    print(f"[{timestamp}] Processing event: {event_type}")
    
    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])  # Trigger Left Poke signal
        print("Left poke event triggered.")
        
    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])  # Trigger Right Poke signal
        print("Right poke event triggered.")
        
    elif event_type == "LeftWithPellet":
        send_ttl_signal(gpio_pins["LeftWithPellet"])  # Trigger LeftWithPellet signal briefly
        print("Left poke with pellet, signal triggered.")
        
    elif event_type == "RightWithPellet":
        send_ttl_signal(gpio_pins["RightWithPellet"])  # Trigger RightWithPellet signal briefly
        print("Right poke with pellet, signal triggered.")
        
    elif event_type in ["Pellet", "PelletInWell"]:
        handle_pellet_event(event_type)  # Handle PelletInWell and PelletTaken

# Function to read from serial port (FED3 devices)
def read_from_fed(serial_port):
    ser = serial.Serial(serial_port, 115200, timeout=1)
    while True:
        line = ser.readline().decode('utf-8').strip()
        if line:
            data_list = line.split(",")  # Split the data string into a list
            print(f"Raw FED3 data: {data_list}")  # Debug: Show the full data received
            if len(data_list) == len(column_headers):  # Ensure the data matches the column length
                event_type = data_list[9]  # "Event" field contains event type
                timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]  # Get current timestamp
                print(f"Extracted event type: {event_type} at {timestamp}")
                process_event(event_type, timestamp)  # Send TTL signal for the event
            else:
                print("Warning: Data length does not match header length, skipping.")

# Define serial ports for each FED device
fed_ports = {
    "/dev/ttyACM0": "FED1",  # Add more FED ports if needed
}

# Start threads to handle multiple FED devices
for port in fed_ports.keys():
    threading.Thread(target=read_from_fed, args=(port,)).start()

# Cleanup GPIO when script ends
try:
    while True:
        time.sleep(1)  # Keep the script running
except KeyboardInterrupt:
    print("Cleaning up GPIO and exiting.")
    GPIO.cleanup()


# ChatGPT 01-preview has some suggestions to make sure the code works well when multiple FEDs are included, first I am trying to test the code suggested by ChatGPT


### Key Changes and Improvements:

1. **Thread Safety with Global Variables**:
   - Introduced a `pellet_lock` (`threading.Lock()`) to ensure thread-safe access to the `pellet_in_well` variable.
   - Wrapped access to `pellet_in_well` within `with pellet_lock:` blocks.

2. **Exception Handling for Serial Communication**:
   - Added `try...except` blocks around serial port operations to catch and log exceptions like `serial.SerialException`.
   - Ensured that the serial port is closed properly in the `finally` block.

3. **Graceful Shutdown and GPIO Cleanup**:
   - Used a `stop_event` (`threading.Event()`) to signal threads to stop when a `KeyboardInterrupt` occurs.
   - In the main thread, upon interruption, set the `stop_event` and joined all threads before cleaning up GPIO pins.

4. **Timestamp Accuracy**:
   - Modified the `process_event` function to use the timestamp from the FED3 data (`data_list[0]`) instead of generating a new timestamp. This ensures more accurate event timing.

5. **Data Validation and Parsing**:
   - Added checks to ensure that the data list has at least 10 elements before accessing `data_list[9]` to avoid `IndexError`.
   - Logged warnings when received data is incomplete or does not match expected length.

6. **Resource Management for Serial Ports**:
   - Ensured that serial ports are closed when threads terminate, using `ser.close()` in the `finally` block.

7. **Logging Instead of Print Statements**:
   - Replaced all `print` statements with `logging` calls, providing more control over log levels and formatting.
   - Configured the logging format to include timestamps and log levels.

8. **Error Handling in Threads**:
   - Wrapped the main logic in the `read_from_fed` function within a `try...except` block to catch and log any exceptions that occur within the thread.

9. **Handling Multiple FED Devices**:
   - The code is structured to handle multiple FED devices by starting a new thread for each serial port specified in the `fed_ports` dictionary.
   - Each thread handles its own serial port and shared resources are managed with thread-safe mechanisms.

10. **Graceful Thread Termination**:
    - Threads check the `stop_event` flag in their loops and exit gracefully when the event is set.

11. **Ensuring GPIO Cleanup**:
    - Moved `GPIO.cleanup()` into the `finally` block to ensure it executes regardless of how the script exits.

12. **Code Organization and Modularity**:
    - Organized the code into functions for better readability and maintainability.

### Additional Notes:

- **Serial Port Detection**:
  - The code still uses hardcoded serial ports. For a more robust solution, consider implementing dynamic detection of connected FED devices or using persistent device naming (e.g., by device ID or symbolic links).

- **Exception Handling**:
  - In the `read_from_fed` function, exceptions are logged but the thread continues running unless a critical error occurs, ensuring robustness.

- **Logging Levels**:
  - Adjust the logging level in `logging.basicConfig()` to control the verbosity of the logs (`DEBUG`, `INFO`, `WARNING`, `ERROR`, `CRITICAL`).

- **Python Version**:
  - Added the shebang line `#!/usr/bin/env python3` at the top of the script to specify that it should be run with Python 3.

- **Thread Cleanup**:
  - The main thread waits for all worker threads to finish by calling `t.join()` on each thread after setting the `stop_event`.

- **Resource Management**:
  - The serial port is closed in the `finally` block, ensuring that it is closed even if an error occurs during execution.

Please replace `/dev/ttyACM0` and `FED1` in the `fed_ports` dictionary with the actual serial ports and identifiers for your devices.


In [ ]:

#!/usr/bin/env python3

import RPi.GPIO as GPIO
import serial
import threading
import datetime
import time
import logging
import sys
import traceback

# Configure logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s [%(levelname)s] %(message)s',
    stream=sys.stdout
)

# Setup GPIO pins on the Raspberry Pi (BCM mode)
GPIO.setmode(GPIO.BCM)
GPIO.setwarnings(False)

# Define GPIO pins for the TTL output
gpio_pins = {
    "LeftPoke": 17,          # Pin for Left poke event (GPIO 17, Physical Pin 11)
    "RightPoke": 27,         # Pin for Right poke event (GPIO 27, Physical Pin 13)
    "LeftWithPellet": 22,    # Pin for Left poke with pellet event (GPIO 22, Physical Pin 15)
    "RightWithPellet": 23,   # Pin for Right poke with pellet event (GPIO 23, Physical Pin 16)
    "Pellet": 24,            # Pin for PelletInWell and PelletTaken (GPIO 24, Physical Pin 18)
}

# Set all pins as output and initially set them to LOW
for pin in gpio_pins.values():
    GPIO.setup(pin, GPIO.OUT)
    GPIO.output(pin, GPIO.LOW)

# Create a lock for thread-safe access to shared variables
pellet_lock = threading.Lock()

# Track the state of Pellet in Well
pellet_in_well = False  # Keeps track of whether a pellet is in the well

# Define the column headers based on your desired CSV structure
column_headers = [
    "MM/DD/YYYY hh:mm:ss.SSS", "Temp", "Humidity", "Library_Version", "Session_type",
    "Device_Number", "Battery_Voltage", "Motor_Turns", "FR", "Event", "Active_Poke",
    "Left_Poke_Count", "Right_Poke_Count", "Pellet_Count", "Block_Pellet_Count",
    "Retrieval_Time", "InterPelletInterval", "Poke_Time"
]

# Global event to signal threads to stop
stop_event = threading.Event()

# Function to send TTL pulse for regular poke events
def send_ttl_signal(pin):
    logging.debug(f"Sending TTL signal to pin {pin}")
    GPIO.output(pin, GPIO.HIGH)
    time.sleep(0.01)  # Send a 10 ms pulse
    GPIO.output(pin, GPIO.LOW)

# Function to handle the PelletInWell/PelletTaken logic
def handle_pellet_event(event_type):
    global pellet_in_well
    with pellet_lock:
        if event_type == "Pellet":
            if pellet_in_well:
                GPIO.output(gpio_pins["Pellet"], GPIO.LOW)  # Pellet taken, turn the signal off
                logging.debug("Pellet taken, signal turned OFF.")
                pellet_in_well = False  # Update state
                send_ttl_signal(gpio_pins["Pellet"])  # Send a short TTL pulse for PelletTaken
            else:
                logging.debug("No pellet was in the well, no signal for pellet taken.")
        elif event_type == "PelletInWell":
            GPIO.output(gpio_pins["Pellet"], GPIO.HIGH)  # Pellet in well, keep signal on
            pellet_in_well = True
            logging.debug("Pellet dispensed in well, signal ON.")

# Function to process each event and send TTLs accordingly
def process_event(event_type):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S.%f")[:-3]
    logging.info(f"[{timestamp}] Processing event: {event_type}")

    if event_type == "Left":
        send_ttl_signal(gpio_pins["LeftPoke"])  # Trigger Left Poke signal
        logging.debug("Left poke event triggered.")

    elif event_type == "Right":
        send_ttl_signal(gpio_pins["RightPoke"])  # Trigger Right Poke signal
        logging.debug("Right poke event triggered.")

    elif event_type == "LeftWithPellet":
        send_ttl_signal(gpio_pins["LeftWithPellet"])  # Trigger LeftWithPellet signal briefly
        logging.debug("Left poke with pellet, signal triggered.")

    elif event_type == "RightWithPellet":
        send_ttl_signal(gpio_pins["RightWithPellet"])  # Trigger RightWithPellet signal briefly
        logging.debug("Right poke with pellet, signal triggered.")

    elif event_type in ["Pellet", "PelletInWell"]:
        handle_pellet_event(event_type)  # Handle PelletInWell and PelletTaken

# Function to read from serial port (FED3 devices)
def read_from_fed(serial_port):
    try:
        try:
            ser = serial.Serial(serial_port, 115200, timeout=1)
            logging.info(f"Opened serial port {serial_port}")
        except serial.SerialException as e:
            logging.error(f"Could not open serial port {serial_port}: {e}")
            return
        except Exception as e:
            logging.error(f"Unexpected error opening serial port {serial_port}: {e}")
            return

        while not stop_event.is_set():
            try:
                line = ser.readline().decode('utf-8').strip()
                if line:
                    data_list = line.split(",")
                    logging.debug(f"Raw FED3 data: {data_list}")  # Debug: Show the full data received
                    if len(data_list) >= 10:
                        event_type = data_list[9]  # "Event" field contains event type
                        # Use Raspberry Pi's system time for timestamp
                        logging.debug(f"Extracted event type: {event_type}")
                        process_event(event_type)  # Send TTL signal for the event
                    else:
                        logging.warning("Received incomplete data, skipping this line.")
            except UnicodeDecodeError as e:
                logging.error(f"Decoding error on serial port {serial_port}: {e}")
            except Exception as e:
                logging.error(f"Error reading from serial port {serial_port}: {e}")
                logging.error(traceback.format_exc())
    except Exception as e:
        logging.error(f"Critical error in thread for serial port {serial_port}: {e}")
        logging.error(traceback.format_exc())
    finally:
        try:
            ser.close()
            logging.info(f"Serial port {serial_port} closed.")
        except Exception as e:
            logging.error(f"Error closing serial port {serial_port}: {e}")

# Define serial ports for each FED device
fed_ports = {
    "/dev/ttyACM0": "FED1",  # Add more FED ports if needed
}

threads = []

# Start threads to handle multiple FED devices
for port in fed_ports.keys():
    t = threading.Thread(target=read_from_fed, args=(port,))
    t.start()
    threads.append(t)

# Cleanup GPIO when script ends
try:
    while True:
        time.sleep(1)  # Keep the script running
except KeyboardInterrupt:
    logging.info("Interrupted by user.")
    stop_event.set()  # Signal threads to stop
finally:
    # Wait for threads to finish
    for t in threads:
        t.join()
    logging.info("Cleaning up GPIO and exiting.")
    GPIO.cleanup()


# The code above suggested by ChatGPT works well (in theory it sends signals to RBPI and I can see the LEDs blinking in response to TTLS)